In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [46]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite",echo=True)
this_conn = engine.connect()

2021-07-21 17:22:47,393 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-07-21 17:22:47,395 INFO sqlalchemy.engine.base.Engine ()
2021-07-21 17:22:47,397 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-07-21 17:22:47,399 INFO sqlalchemy.engine.base.Engine ()


In [18]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine)

In [9]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [10]:
### Create the inspector and connect it to the engine
inspector = inspect(engine)

In [11]:
# Save references to each table
### Collect the names of tables within the database
inspector.get_table_names()

2021-07-21 16:28:49,117 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-07-21 16:28:49,118 INFO sqlalchemy.engine.base.Engine ()


['measurement', 'station']

In [53]:
# get columns for measurement table
meas_cols = inspector.get_columns('measurement')
for c in meas_cols:
    print(c['name'], c['type'])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [54]:
# get columns for station table
stat_cols = inspector.get_columns('station')
for c in stat_cols:
    print(c['name'], c['type'])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [51]:
# not sure I need these, but doing it because it was in very first activity Query All Records in the Database
measurement_data = engine.execute("SELECT * FROM measurement")
measurement_df = pd.read_sql("SELECT * FROM measurement", this_conn)

measurement_df.head()

2021-07-21 17:25:30,796 INFO sqlalchemy.engine.base.Engine SELECT * FROM measurement
2021-07-21 17:25:30,798 INFO sqlalchemy.engine.base.Engine ()
2021-07-21 17:25:30,801 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM measurement")
2021-07-21 17:25:30,802 INFO sqlalchemy.engine.base.Engine ()
2021-07-21 17:25:30,803 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM measurement")
2021-07-21 17:25:30,804 INFO sqlalchemy.engine.base.Engine ()
2021-07-21 17:25:30,806 INFO sqlalchemy.engine.base.Engine SELECT * FROM measurement
2021-07-21 17:25:30,809 INFO sqlalchemy.engine.base.Engine ()


,id,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65.0
1,2,USC00519397,2010-01-02,0.00,63.0
2,3,USC00519397,2010-01-03,0.00,74.0
3,4,USC00519397,2010-01-04,0.00,76.0
4,5,USC00519397,2010-01-06,NaN,73.0


In [49]:
# not sure I need these, but doing it because it was in very first activity Query All Records in the Database
station_data = engine.execute("SELECT * FROM station")
station_df = pd.read_sql("SELECT * FROM station", this_conn)

station_df

2021-07-21 17:23:52,708 INFO sqlalchemy.engine.base.Engine SELECT * FROM station
2021-07-21 17:23:52,710 INFO sqlalchemy.engine.base.Engine ()
2021-07-21 17:23:52,718 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM station")
2021-07-21 17:23:52,719 INFO sqlalchemy.engine.base.Engine ()
2021-07-21 17:23:52,722 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM station")
2021-07-21 17:23:52,723 INFO sqlalchemy.engine.base.Engine ()
2021-07-21 17:23:52,725 INFO sqlalchemy.engine.base.Engine SELECT * FROM station
2021-07-21 17:23:52,726 INFO sqlalchemy.engine.base.Engine ()


,id,station,name,latitude,longitude,elevation
0,1,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,2,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,3,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,4,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,5,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,6,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,7,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,8,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,9,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [12]:
### Assign classes to variables 
Measurement = Base.classes.measurement
Station = Base.classes.station

In [32]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [58]:
# Find the most recent date in the data set.
### Latest Date
### isn't a last func so have to sort desc & use .first()
earliest_date = session.query(Measurement.date).order_by(Measurement.date).first()
print(earliest_date)

2021-07-21 17:34:42,501 INFO sqlalchemy.engine.base.Engine SELECT measurement.date AS measurement_date 
FROM measurement ORDER BY measurement.date
 LIMIT ? OFFSET ?
2021-07-21 17:34:42,505 INFO sqlalchemy.engine.base.Engine (1, 0)
('2010-01-01',)


In [59]:
last_date = session.query(Measurement.date).order_by(Measurement.date.desc).first()
print(last_date)

ArgumentError: SQL expression object expected, got object of type <class 'method'> instead

## Design a query to retrieve the last 12 months of precipitation data and plot the results. 

In [ ]:
# Starting from the most recent data point in the database. 
end_date = last_date
# Calculate the date one year from the last date in data set.
start_date = last_date + 365 # guessing this won't work, but need to look at documentation, just wanted a conceptual placeholder.

sel = [Dow.stock, 
       func.avg(Dow.open_price), 
       func.avg(Dow.high_price), 
       func.avg(Dow.low_price), 
       func.avg(Dow.close_price)]
may_averages = session.query(*sel).\
    filter(func.strftime("%m", Dow.date) == "05").\
    group_by(Dow.stock).\
    order_by(Dow.stock).all()
may_averages

In [23]:
# Perform a query to retrieve the data and precipitation scores
### is there a date between function?
sel = [Measurement.date, Measurement.prcp]
precip_output = session.query('SELECT sel WHERE TABLENAME.date>=start_date).order_by(TABLENAME.date()')

In [24]:
# Save the query results as a Pandas DataFrame and set the index to the date column
precipitation_df = pd.DataFrame(precip_output)

2021-07-21 16:39:33,023 INFO sqlalchemy.engine.base.Engine SELECT SELECT sel WHERE TABLENAME.date>=start_date).order_by(TABLENAME.date()
2021-07-21 16:39:33,027 INFO sqlalchemy.engine.base.Engine ()


OperationalError: (sqlite3.OperationalError) near "SELECT": syntax error
[SQL: SELECT SELECT sel WHERE TABLENAME.date>=start_date).order_by(TABLENAME.date()]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [ ]:
# Sort the dataframe by date
precipitation_df = precipitation_df.sortby['date']

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precip_summary_df = precipitation_df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()